In [45]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import h5py
import copy
from scipy.optimize import curve_fit

In [46]:
class Sig:
    def __init__(self):
        self.number_groups = 0
        self.sig_f = []
        self.capture = []
        self.scattering = []
        self.total = []
        self.number_of_production_neutrons = []
        self.c_value = []
        self.virtual = []
        
    def __init__(self, number_groups, sig_f, sig_c, sig_s, number_of_production_neutrons, sig_t):
        self.number_groups = number_groups
        self.sig_f = sig_f
        self.sig_c = sig_c
        self.sig_s = sig_s      
        self.number_of_production_neutrons = number_of_production_neutrons
        self.sig_t = sig_t
        self.delta_xs = max(sig_t)/2.
        max_total_cs = max(self.sig_t)
        self.virtual = [self.delta_xs]
        
               
    def get_virtual_cs(self):
        max_total_cs = max(self.sig_t)
        self.virtual = [cross_section - max_total_cs + self.delta_xs for cross_section in self.sig_t]
        
    def return_virtual(self):
        return self.virtual
        
    def get_fission_probability(self, energy_group_idx):
        return self.sig_f[energy_group_idx] / self.sig_t[energy_group_idx]
    
    def get_capture_probability(self, energy_group_idx):
        return self.sig_c[energy_group_idx] / self.sig_t[energy_group_idx]
        
    def get_scatter_probability(self, energy_group_idx):
        return self.sig_s[energy_group_idx] / self.sig_t[energy_group_idx]
        

In [47]:
energy_groups_2 = [0, 2 * 1E10]

class Positon:
    def __init__(self, x, y, z):
        self.x = x
        self.y = y
        self.z = z
        
class Direction:
    def __init__(self, ets, phi):
        self.tetta_x = math.sin(ets) * math.cos(phi)
        self.tetta_y = math.sin(ets) * math.sin(phi)
        self.tetta_z = math.cos(ets)  

In [48]:
class Particle:
    def __init__(self):
        self.coordinates = Positon(0, 0, 0)
        self.direction = Direction(0, 0)
        self.energy = 0
        self.weight = 1
        self.energy_groups = []
        self.terminated = False
        self.path = 0.
        self.last_cell = 0
        
    def set_coordinates(self, x, y, z):
        self.coordinates = Positon(x, y, z)
        
    def set_direction(self, ets, phi):
        self.direction = Direction(ets, phi)
    
    def set_direction_angle(self, angle):
        self.direction = angle
        
    def set_energy_groups(self, energy_groups):
        self.energy_groups = energy_groups
        
    def get_energy_group(self):  
        res = next(x for x, val in enumerate(self.energy_groups)
                                  if val > self.energy)        
        return res - 1
    
    def set_terminated(self):
        self.terminated = True
        
    def is_terminated(self):
        return self.terminated
        
    def get_weight(self):
        return self.weight
    
    def set_weight(self, weight):
        self.weight = weight
    
    def set_particle_deleted(self):
        self.weight = self.weight * 0
        
    def set_particle_fission(self, additional_weight):
        self.weight = self.weight * additional_weight
        
    def set_multiplicity(self, additional_weight):
        self.weight = self.weight * additional_weight
              
    def is_particle_deleted(self):
        return self.weight < 0.0001
    
    def add_path(self, path):
        self.path += path
    
    def get_path(self):
        return self.path
    
    def set_path(self, path):
        self.path = path
        
    def print_direction(self):
        print(str(self.direction.tetta_x) + " " + str(self.direction.tetta_y) + " " + str(self.direction.tetta_z))
        
    def print_coordinates(self):
        print(str(self.coordinates.x) + "  " + str(self.coordinates.y) + " " + str(self.coordinates.z))  
        

In [49]:
class Plane:
    def __init__(self, A, B, C, D):
        self.A = A
        self.B = B
        self.C = C
        self.D = D
        
    def distance(self, particle):
        vp = (self.A * particle.direction.tetta_x + self.B * particle.direction.tetta_y + 
              self.C * particle.direction.tetta_z)
        
        
        if (abs(vp) < 1e-9):
            return -1
        
        distance = (self.D - self.A * particle.coordinates.x - self.B * particle.coordinates.y 
                    - self.C * particle.coordinates.z) / vp
        
        return distance 
    
    def get_normal(self):
        sq = math.sqrt(self.A * self.A + self.B * self.B + self.C * self.C)
        a_n = self.A / sq
        b_n = self.B / sq
        c_n = self.C / sq
        return [a_n, b_n, c_n]
    

    def get_sign(self, particle):
      
        sign = (self.A * particle.coordinates.x + self.B * particle.coordinates.y +
        self.C * particle.coordinates.z - self.D)
        
        if sign == 0:
            return sign    
            
        if sign < 0:
            return -1
        else:
            return 1
        
    def get_xml(self):
        
        obj_xml = " Plane with A "+ str(self.A) + " B "+ str(self.B) + " C "+ str(self.C) + " D "+ str(self.D)
        
        return obj_xml
    

In [50]:
class Universe:
    def __init__(self, cells, xs):
        self.cells = cells
        self.xs = xs
        self.delta_xs = 0.
        
    def calculate_delta_xs(self):
        
        delta_xs = 0.
        for xs in self.xs:
            delta_xs = max(delta_xs, max(xs.sig_t))
        self.delta_xs = delta_xs           
            
    def get_delta_xs(self):
        
        return self.delta_xs
    
    def get_xs_by_coordinates(self, particle):
        
        for i in range(0, len(cells)):
            cell = cells[i]
            if cell.is_inside(particle):
                particle.last_cell = i
                return self.xs[i]
        return []
    
    def is_inside(self, particle):
        
        for cell in self.cells:
            if cell.is_inside(particle):
                return True
        return False
        
    
    def is_boudary_reflective(self, particle):
        
        minumum = np.inf
        idx_of_min = 0.
        for i in range(0, len(self.cells)):
            distance, surface_idx = self.cells[i].get_minimum_distance(particle)
            if abs(distance) < minumum:
                minumum = distance
                idx_of_min = i
        if self.cells[idx_of_min].is_boudary_reflective(particle):
            return True
        else:
            return False
        
    def reflect_particle(self,  particle):
        self.cells[particle.last_cell].reflect_particle(particle)       


In [51]:
def set_random_direction(particle):
    r = 1.0
    r1 = 1.0
    while (r**2 + r1**2 > 1.0):
        r = 2. * random.uniform(0, 1) - 1.
        r1 = 2. * random.uniform(0, 1) - 1.
        
    direction_x = 2.0 * r**2 + 2. * r1**2 - 1.0  
    particle.direction.tetta_x = direction_x
    particle.direction.tetta_y = r * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))
    particle.direction.tetta_z = r1 * math.sqrt((1.0 - direction_x**2)/(r**2+r1**2))

In [52]:
import random

def make_initial_sources(number_of_paricles, box_size, zero_point, energy=10.0e6):
    
    step_x = box_size[0]/number_of_paricles
    step_y = box_size[1]/number_of_paricles
    step_z = box_size[2]/number_of_paricles
    
    x_coord = zero_point[0]
    y_coord = zero_point[1]
    z_coord = zero_point[2]
    
    sources = []
    for i in range(0, number_of_paricles):
        for j in range(0, number_of_paricles):
            for k in range(0, number_of_paricles):
                current_particle = Particle()
                current_particle.set_coordinates(x_coord + k * step_x, y_coord + j * step_y, z_coord+ i * step_z)
                set_random_direction(current_particle)
                current_particle.energy = energy
                current_particle.set_energy_groups(energy_groups_2)
                sources.append(current_particle)

    return sources

In [53]:
def get_free_path(particle, delta_xs):
    
    energy_group_idx = particle.get_energy_group()   
    sig_t = delta_xs
    free_path = -math.log(random.uniform(0, 1)) / sig_t
    
    return free_path  

In [54]:
class Cell:
    def __init__(self):
        self.surfaces = []
        self.boundaries_type = []
        self.signs = []
        self.size = 0
        
    def set_boundaries_type(self, boundaries_type):
        self.boundaries_type = boundaries_type        
        
    def set_box_sizes(self, xm_size, ym_size, zm_size, x_size, y_size, z_size):
        self.xm_size = xm_size
        self.ym_size = ym_size
        self.zm_size = zm_size
        
        self.x_size = x_size
        self.y_size = y_size
        self.z_size = z_size
        
    def set_zero_point(self, x_0, y_0, z_0):
        self.x_0 = x_0
        self.y_0 = y_0
        self.z_0 = z_0
        
    def __init__(self, surfaces, signs):
        self.surfaces = surfaces
        self.signs = signs
        self.size = len(self.signs)
           
    def is_inside(self, particle):
        
        is_inside = True
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                is_inside = False
                
        return is_inside
    
    def is_boudary_reflective(self, particle):
        for i in range(0, self.size):
            sign = self.surfaces[i].get_sign(particle)
            if self.signs[i] != sign:
                if self.boundaries_type[i] == 'reflective':
                    return True
                else:
                    return False
        
    
    def get_minimum_distance(self, particle):
        
        minimum_distance = math.inf
        surface_idx = 0
        for i in range(0, len(self.surfaces)):
            current_distance = abs(self.surfaces[i].distance(particle))
            if current_distance > 0:
                minimum_distance = min(minimum_distance, current_distance)
                surface_idx = i
        return minimum_distance, surface_idx
    
    def get_xml(self):
        
        xml_obj = []
        for i in range(0, self.size):
            xml_obj.append(surfaces[i].get_xml())
            
        return xml_obj
    
    def reflect_particle(self, particle):
        
        if particle.coordinates.x > (self.x_size + self.x_0):
            delta = particle.coordinates.x % self.x_size
            particle.coordinates.x = self.xm_size + math.fabs(delta)
        
        if particle.coordinates.y > (self.y_size + self.y_0):
            
            delta = particle.coordinates.y % self.y_size
            particle.coordinates.y = self.ym_size + math.fabs(delta)
            
            
        if particle.coordinates.z > (self.z_size + self.z_0):
            
            delta = particle.coordinates.z % self.z_size
            particle.coordinates.z = self.zm_size + math.fabs(delta)
        
        
        if particle.coordinates.y < (self.ym_size + self.y_0):
            delta = particle.coordinates.y % self.ym_size
            particle.coordinates.y = self.y_size - math.fabs(delta)
            
            
        if particle.coordinates.x < (self.xm_size + self.x_0):
            delta = particle.coordinates.x % self.xm_size
            particle.coordinates.x = self.x_size - math.fabs(delta)
            
        if particle.coordinates.z < (self.zm_size + self.z_0):
            delta = particle.coordinates.z % self.zm_size
            particle.coordinates.z = self.z_size - math.fabs(delta)
                    

In [55]:
def move_particle(particle, t):
    new_x = particle.direction.tetta_x * t + particle.coordinates.x
    new_y = particle.direction.tetta_y * t + particle.coordinates.y
    new_z = particle.direction.tetta_z * t + particle.coordinates.z
    particle.set_coordinates(new_x, new_y, new_z)

In [56]:
def is_collision_virual(particle, c_s, delta_xs):
    
    energy_group_idx = particle.get_energy_group()
    random_number = random.uniform(0, 1)
    
    virtual_cs = delta_xs
    
    total_cs = c_s.sig_t[energy_group_idx]
    
    if total_cs/virtual_cs < random_number:
        return True
    else:
        return False

In [57]:
def process_virtual_collision(particle, free_path):
    particle.add_path(free_path)

In [58]:
def process_real_collision(particle, free_path, c_s):
    
    energy_group_idx = particle.get_energy_group()
    weight_before_collision = particle.get_weight()
    number_of_production_neutrons = c_s.number_of_production_neutrons[energy_group_idx]   
    capture_probability = c_s.get_capture_probability(energy_group_idx)
    scatter_probability = c_s.get_scatter_probability(energy_group_idx)  
    fission_probability = c_s.get_fission_probability(energy_group_idx)  
    
    
    type_collision = np.random.choice(['capture', 'scatter', 'fission'], p=[capture_probability, scatter_probability,
                                                                            fission_probability])
    if type_collision == 'capture':
        particle.set_terminated()
        particle.set_weight(0.)
        
    if type_collision == 'scatter':      
        set_random_direction(particle)
        
    if type_collision == 'fission':    
        particle.set_particle_fission(number_of_production_neutrons)
        particle.set_terminated()
        
    return weight_before_collision

In [59]:
def delete_absorpbed_paricles(particles):
    
    existing_particles = []
    for i in range(0, len(particles)):
        particle = particles[i]
        if particle.get_weight() > 0.000001:
            existing_particles.append(particle)
                     
    return existing_particles

In [60]:
def process_one_particle_history(particle, universe, estimators):
    
    sum_collisions = 0.
    universe.calculate_delta_xs()
    delta_xs = universe.get_delta_xs()
    
    while not particle.is_terminated():
        set_random_direction(particle)
        current_xs = universe.get_xs_by_coordinates(particle)
        free_path = get_free_path(particle, delta_xs)        
        move_particle(particle, free_path)
        current_xs = universe.get_xs_by_coordinates(particle)
        
        if not universe.is_inside(particle):
            if universe.is_boudary_reflective(particle):
                universe.reflect_particle(particle)
                current_xs = universe.get_xs_by_coordinates(particle)

            else:
                particle.set_terminated()
                particle.set_weight(0.)
                return particle, sum_collisions
                
        for k in estimators:
            k.add_collision(particle)
        
        if is_collision_virual(particle, current_xs, delta_xs):
            process_virtual_collision(particle, free_path)
        else:
            process_real_collision(particle, free_path, current_xs)
        
            
    return particle, sum_collisions

In [61]:
def get_weights(particles):
    
    weights = []
    for i in range(0, len(particles)):
        weights.append(particles[i].weight)
        
    return weights

In [62]:
def make_sources(particles):
    for i in range(0, len(particles)):
        particles[i].terminated = False
        set_random_direction(particles[i])       

In [63]:
def splitting_secound_version(particles):
    
    initial_size = len(particles)
    
    for i in range(0, initial_size):
        particle = particles[i]
        current_weight = particle.get_weight()
        if current_weight > 1:
            n_value = math.floor(current_weight)
            random_value = random.uniform(0, 1)
            if current_weight - n_value >= random_value:
                n_value += 1
            if n_value > 1:
                particle.set_weight(current_weight / n_value)
                for j in range(0, n_value - 1):
                    particles.append(copy.deepcopy(particle))

In [64]:
def normalise_weights(particles, batch_size):

    sum_weights = 0.
    
    for i in range(0, len(particles)):
        sum_weights += particles[i].get_weight()

    for i in range(0, len(particles)):
        particles[i].weight = (particles[i].weight * batch_size) / sum_weights   
        
    return particles

In [65]:
def russian_roulette(weights_previous, particles_current):
    
    for i in range(0, len(particles_current)):
        particle = particles_current[i]
        if particle.weight > 0 and particle.weight < 0.5 and particle.weight < weights_previous[i]:
            probability_terminate = 1. - particle.weight / weights_previous[i]
            random_number = random.uniform(0, 1)
            if probability_terminate >= random_number:
                particle.set_weight(0.)
            else:
                particle.set_weight(weights_previous[i])

In [66]:
def terminate_outside_particles(batch_particles, universe):
    
    for j in range(0, len(batch_particles)):
        particle = batch_particles[j]
        if not universe.is_inside(particle):
            print("ERROR! outside particle")

In [67]:
def reset_estimators(estimators, weight_previos, volume, sum_collisions, c_s):
    
    sig_t = c_s.sig_t[0]

    for estimator in estimators:
        estimator.collision_sum[-1] = estimator.collision_sum[-1]
        estimator.collision_sum.append(0.)  

In [68]:
def get_std(values):
    
    current_std = np.std(values)/np.sqrt((len(values) - 1.))
  
    return current_std   

In [69]:
def calculate_k_effective(idx, weights, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, std_k_effective):
    
    keff_cycle = sum(weights) / initial_size
    if idx > number_inactive:
        k_effective.append(keff_cycle)
        
    if idx > number_inactive + 1:
    
        k_effective_exp.append(sum(k_effective) / len(k_effective))
        
        std_k_effective_current = get_std(k_effective)
            
        std_k_effective.append(std_k_effective_current)

        print(" keff_cycle , k_effective_exp, std_k_effective " + str(keff_cycle) + "   " + str(k_effective_exp[-1]) +
                                    "  "+ str(std_k_effective[-1]))

In [70]:
import statistics
import time


def simulation_black_boundaries(universe, number_interations, number_inactive, number_of_particles, c_s, estimators,
                                volume=1):
    
    random.seed(time.time())
    k_effective = []
    k_effective_std = []
    k_effective_exp = []
    flux = []
    flux_exp = []
    num1 = random.randint(0,9)
    
    print(" num1  " + str(num1))
    
    box_size = [1., 1., 1.]
    zero_point = [0.1, 0.1, 0.1]
    initial_sources = make_initial_sources(number_of_particles, box_size, zero_point, energy=10.0e6)
    initial_size = len(initial_sources)
    
    weights_previous = [1.] * len(initial_sources)
    
    for i in range(0, number_interations):
        print("i == " + str(i))
        
        make_sources(initial_sources)
        batch_size = len(initial_sources)
        
        batch_particles = []

        for j in range(0, batch_size):
   
            particle = initial_sources[j]
            terminate_particle, sum_collisions = process_one_particle_history(particle, universe, estimators)
            batch_particles.append(terminate_particle)
        
        russian_roulette(weights_previous, batch_particles)
        
        batch_particles = delete_absorpbed_paricles(batch_particles)
        
        splitting_secound_version(batch_particles)
        
        weights_cycle = get_weights(batch_particles)
        
        calculate_k_effective(i, weights_cycle, number_interations, number_inactive, initial_size, k_effective,
                         k_effective_exp, k_effective_std)
        
        
        reset_estimators(estimators, weights_previous, volume, sum_collisions, c_s) 
      

        batch_particles = normalise_weights(batch_particles, initial_size)  

        initial_sources = batch_particles
        weights_previous = get_weights(batch_particles)

            
    return k_effective, k_effective_exp, k_effective_std, estimators

In [71]:
cs_fission_u235 = [0.06922744]

cs_capture_u235 = [0.01013756]

cs_scattering_u235 = [0.328042]

cs_total_u235 = [0.407407]

cs_production_neutrons_u235 = [2.50]


u235 = Sig(1, cs_fission_u235, cs_capture_u235, cs_scattering_u235, 
                       cs_production_neutrons_u235,  cs_total_u235)


cs_fission_fe = [0.]

cs_capture_fe = [0.00046512]

cs_scattering_fe = [0.23209488]

cs_total_fe = [0.23256]

cs_production_neutrons_fe = [0.]

fe = Sig(1, cs_fission_fe, cs_capture_fe, cs_scattering_fe, 
                       cs_production_neutrons_fe,  cs_total_fe)


cs_fission_na = [0.]

cs_capture_na = [0.0]

cs_scattering_na = [0.086368032]

cs_total_na = [0.086368032]

cs_production_neutrons_na = [0.]

na = Sig(1, cs_fission_na, cs_capture_na, cs_scattering_na, 
                       cs_production_neutrons_na,  cs_total_na)



energy_groups_2 = [0, 2 * 1E10]

In [72]:
x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 0.317337461)

y_m5 = Plane(0, 1, 0, 0.)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab_fe = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective","black", "black", "reflective", "reflective"]


slab_fe.set_boundaries_type(boundaries_type)
slab_fe.set_box_sizes(-1000, 10900., -1000, 1000, 20000., 1000 )
slab_fe.set_zero_point(0., 0., 0.)

In [73]:
x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 5.437057544)

y_m5 = Plane(0, 1, 0, 0.317337461)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab_u235 = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective","black", "black", "reflective", "reflective"]


slab_u235.set_boundaries_type(boundaries_type)
slab_u235.set_box_sizes(-1000, -10000., -1000, 1000, 10000., 1000 )
slab_u235.set_zero_point(0., 0., 0.)

In [74]:
x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 5.754395005)

y_m5 = Plane(0, 1, 0, 5.437057544)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab_fe2 = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective","black", "black", "reflective", "reflective"]


slab_fe2.set_boundaries_type(boundaries_type)
slab_fe2.set_box_sizes(-1000, -10000., -1000, 1000, 10000., 1000 )
slab_fe2.set_zero_point(0., 0., 0.)

In [75]:
x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 7.757166007)

y_m5 = Plane(0, 1, 0, 5.754395005)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
slab_na = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective","black", "black", "reflective", "reflective"]


slab_na.set_boundaries_type(boundaries_type)
slab_na.set_box_sizes(-1000, -10000.,  -1000, 1000, 10000., 1000 )
slab_na.set_zero_point(0., 0., 0.)

In [76]:
cells = [slab_fe, slab_u235, slab_fe2, slab_na]

In [77]:
estimators = []

In [78]:
materials = [fe, u235, fe, na]

In [79]:
U235_Reactor = Universe(cells, materials)

In [80]:
test_number_of_particles = 10
test_number_interations = 500
test_number_inactive = 100

In [81]:
k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(U235_Reactor,  
                           test_number_interations,  test_number_inactive, 
                          test_number_of_particles, u235, estimators)

 num1  6
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
i == 53
i == 54
i == 55
i == 56
i == 57
i == 58
i == 59
i == 60
i == 61
i == 62
i == 63
i == 64
i == 65
i == 66
i == 67
i == 68
i == 69
i == 70
i == 71
i == 72
i == 73
i == 74
i == 75
i == 76
i == 77
i == 78
i == 79
i == 80
i == 81
i == 82
i == 83
i == 84
i == 85
i == 86
i == 87
i == 88
i == 89
i == 90
i == 91
i == 92
i == 93
i == 94
i == 95
i == 96
i == 97
i == 98
i == 99
i == 100
i == 101
i == 102
 keff_cycle , k_effective_exp, std_k_effective 1.181982999332677   1.155815264640493  0.02616773469218403
i == 103
 keff_cycle , k_effective_exp, std_k_effective 1.1455556833

KeyboardInterrupt: 

In [38]:
def save_simulation_results_to_file(k_effective, file_name):
    
    f = open(file_name, "w")
    
    for i in range(0, len(k_effective)):
        f.write(" cycle k effective "+ str(k_effective[i]) + '\n')
            
    
    f.close()

In [39]:
file_name = "output.txt"
save_simulation_results_to_file(k_effective, file_name)

In [40]:
difference = (1.  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  -11430.911936054634  with standart deviation [pcm]+- 206.65539882281885


In [75]:
difference = (1.  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  30.077163825470432  with standart deviation [pcm]+- 130.64752989492595


In [72]:
difference = (1.  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  75.72173614430744  with standart deviation [pcm]+- 202.87392160631452


In [69]:
difference = (1.  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  -75.31115869450211  with standart deviation [pcm]+- 202.14934011690087


In [106]:
difference = (1.  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  66.88487069507198  with standart deviation [pcm]+- 238.37539223781897


In [213]:
x_5 = Plane(1, 0, 0, 1000)
x_m5 = Plane(1, 0, 0, -1000)

y_5 = Plane(0, 1, 0, 1000)

y_m5 = Plane(0, 1, 0,  -1000)


z_5 = Plane(0, 0, 1, 1000)
z_m5 = Plane(0, 0, 1, -1000)

surfaces = [x_5, x_m5, y_5, y_m5, z_5, z_m5]

signs = [-1, +1, -1, +1, -1, +1]
inf_cell = Cell(surfaces, signs)

boundaries_type = ["reflective", "reflective","reflective", "reflective", "reflective", "reflective"]


inf_cell.set_boundaries_type(boundaries_type)
inf_cell.set_box_sizes(-1000, -1000., -1000, 1000, 1000., 1000 )
inf_cell.set_zero_point(0., 0., 0.)

In [244]:
material = [u235]
cells = [inf_cell]

In [245]:
inf_media = Universe(cells, material)

In [246]:
k_effective, k_effective_exp, k_effective_std, estimators_result = simulation_black_boundaries(inf_media,  
                           test_number_interations,  test_number_inactive, 
                          test_number_of_particles, u235, estimators)

 num1  0
i == 0
i == 1
i == 2
i == 3
i == 4
i == 5
i == 6
i == 7
i == 8
i == 9
i == 10
i == 11
i == 12
i == 13
i == 14
i == 15
i == 16
i == 17
i == 18
i == 19
i == 20
i == 21
i == 22
i == 23
i == 24
i == 25
i == 26
i == 27
i == 28
i == 29
i == 30
i == 31
i == 32
i == 33
i == 34
i == 35
i == 36
i == 37
i == 38
i == 39
i == 40
i == 41
i == 42
i == 43
i == 44
i == 45
i == 46
i == 47
i == 48
i == 49
i == 50
i == 51
i == 52
 keff_cycle , k_effective_exp, std_k_effective 2.181787577848804   2.186340154277276  0.004552576428472177
i == 53
 keff_cycle , k_effective_exp, std_k_effective 2.1765545440518888   2.183078284202147  0.0041890866664922085
i == 54
 keff_cycle , k_effective_exp, std_k_effective 2.203478068232153   2.1881782302096484  0.005897768461810088
i == 55
 keff_cycle , k_effective_exp, std_k_effective 2.1654602121349336   2.1836346265947055  0.0064431775939512844
i == 56
 keff_cycle , k_effective_exp, std_k_effective 2.1831292438047876   2.183550396129719  0.005261506733756252
i =

 keff_cycle , k_effective_exp, std_k_effective 2.1947310956926587   2.1842492394432536  0.002028145410415159
i == 120
 keff_cycle , k_effective_exp, std_k_effective 2.1843657225386135   2.1842509034874733  0.0019989626332054277
i == 121
 keff_cycle , k_effective_exp, std_k_effective 2.198677625235373   2.1844540967515282  0.001981055251944456
i == 122
 keff_cycle , k_effective_exp, std_k_effective 2.1385219797431554   2.1838161506819675  0.002054881746057103
i == 123
 keff_cycle , k_effective_exp, std_k_effective 2.150279227318468   2.1833567407728784  0.002077958241908202
i == 124
 keff_cycle , k_effective_exp, std_k_effective 2.1705694518361023   2.1831839395710304  0.0020569565935876716
i == 125
 keff_cycle , k_effective_exp, std_k_effective 2.172571615381447   2.1830424419151693  0.002034272221157701
i == 126
 keff_cycle , k_effective_exp, std_k_effective 2.16707420295312   2.182832333507774  0.002018293175526019
i == 127
 keff_cycle , k_effective_exp, std_k_effective 2.13342193772

 keff_cycle , k_effective_exp, std_k_effective 2.1654703500048518   2.1811705024055654  0.001542754403397016
i == 190
 keff_cycle , k_effective_exp, std_k_effective 2.160641654312681   2.18102386777633  0.0015386980085644792
i == 191
 keff_cycle , k_effective_exp, std_k_effective 2.1928879310344276   2.181108010068941  0.0015300616418748756
i == 192
 keff_cycle , k_effective_exp, std_k_effective 2.1981396981397716   2.181227951534228  0.0015239755587821762
i == 193
 keff_cycle , k_effective_exp, std_k_effective 2.1854861705160697   2.1812577292893462  0.0015135738105395116
i == 194
 keff_cycle , k_effective_exp, std_k_effective 2.1823830647791707   2.1812655441191366  0.0015030464454141084
i == 195
 keff_cycle , k_effective_exp, std_k_effective 2.196035571497888   2.181367406376921  0.0014961162608743469
i == 196
 keff_cycle , k_effective_exp, std_k_effective 2.1644901952444404   2.1812518090403974  0.0014903234855396306
i == 197
 keff_cycle , k_effective_exp, std_k_effective 2.1666502

 keff_cycle , k_effective_exp, std_k_effective 2.1892711620354497   2.180447987936052  0.0011984496413816237
i == 260
 keff_cycle , k_effective_exp, std_k_effective 2.195721382591706   2.1805207183867936  0.0011949445131701117
i == 261
 keff_cycle , k_effective_exp, std_k_effective 2.162218681537448   2.1804339788756595  0.001192426772312538
i == 262
 keff_cycle , k_effective_exp, std_k_effective 2.1778427670059974   2.180421756178161  0.0011868517279905752
i == 263
 keff_cycle , k_effective_exp, std_k_effective 2.148030663494586   2.1802696853204915  0.0011910147434245869
i == 264
 keff_cycle , k_effective_exp, std_k_effective 2.1857309869554493   2.180295205421589  0.0011857108562476847
i == 265
 keff_cycle , k_effective_exp, std_k_effective 2.2002779928214267   2.180388148618798  0.0011838371669238466
i == 266
 keff_cycle , k_effective_exp, std_k_effective 2.1841389788493646   2.180405513573569  0.0011784716378923999
i == 267
 keff_cycle , k_effective_exp, std_k_effective 2.19120406

 keff_cycle , k_effective_exp, std_k_effective 2.182413706223217   2.180945882170952  0.001033363839311023
i == 330
 keff_cycle , k_effective_exp, std_k_effective 2.1926951050566905   2.180987843681258  0.0010305213041382634
i == 331
 keff_cycle , k_effective_exp, std_k_effective 2.1874691500213426   2.1810109088283762  0.001027106432829623
i == 332
 keff_cycle , k_effective_exp, std_k_effective 2.200545317788663   2.181080179782136  0.0010257992938921775
i == 333
 keff_cycle , k_effective_exp, std_k_effective 2.1848963896603593   2.181093664622695  0.001022257079321368
i == 334
 keff_cycle , k_effective_exp, std_k_effective 2.187606942067854   2.1811165986982064  0.0010189093610111395
i == 335
 keff_cycle , k_effective_exp, std_k_effective 2.1907648471982384   2.181150452201715  0.001015892168507667
i == 336
 keff_cycle , k_effective_exp, std_k_effective 2.214164275847343   2.1812658851515248  0.0010188938230708934
i == 337
 keff_cycle , k_effective_exp, std_k_effective 2.195860827834

 keff_cycle , k_effective_exp, std_k_effective 2.1777574510252466   2.181780917011257  0.0009701033518724215
i == 400
 keff_cycle , k_effective_exp, std_k_effective 2.180244428887985   2.1817765270451903  0.0009673376183614666
i == 401
 keff_cycle , k_effective_exp, std_k_effective 2.2040034783773   2.1818398516928603  0.0009666541342512327
i == 402
 keff_cycle , k_effective_exp, std_k_effective 2.1496452824461896   2.1817483898484094  0.0009682335867376146
i == 403
 keff_cycle , k_effective_exp, std_k_effective 2.1511416280609112   2.181661685140796  0.0009693722218413964
i == 404
 keff_cycle , k_effective_exp, std_k_effective 2.189362765834796   2.1816834396060334  0.0009668747698291759
i == 405
 keff_cycle , k_effective_exp, std_k_effective 2.197691609578989   2.181728533042577  0.0009652012722228676
i == 406
 keff_cycle , k_effective_exp, std_k_effective 2.1968195820497214   2.1817709236296756  0.00096341926184314
i == 407
 keff_cycle , k_effective_exp, std_k_effective 2.1856761161

 keff_cycle , k_effective_exp, std_k_effective 2.1707504970179126   2.182146951859748  0.000866166024368892
i == 470
 keff_cycle , k_effective_exp, std_k_effective 2.1848633673339988   2.182153419515639  0.0008641254676602892
i == 471
 keff_cycle , k_effective_exp, std_k_effective 2.1869924024103797   2.1821649135367664  0.0008621470913082636
i == 472
 keff_cycle , k_effective_exp, std_k_effective 2.1667889551076644   2.1821284776163186  0.0008608730718729813
i == 473
 keff_cycle , k_effective_exp, std_k_effective 2.1661469221332332   2.182090696161276  0.0008596661292293746
i == 474
 keff_cycle , k_effective_exp, std_k_effective 2.1639432519466464   2.1820478955852978  0.0008587035414157952
i == 475
 keff_cycle , k_effective_exp, std_k_effective 2.1994068187053677   2.182088740110286  0.0008576538127076427
i == 476
 keff_cycle , k_effective_exp, std_k_effective 2.1921682093714794   2.1821124008362514  0.0008559652515320156
i == 477
 keff_cycle , k_effective_exp, std_k_effective 2.1698

In [247]:
difference = (2.1806667  - k_effective_exp[-1]) * 100000

standart_deviation = k_effective_std[-1] * 100000

print(" difference from beachmark [pcm]  " + str(difference) + "  with standart deviation [pcm]+- " + str(standart_deviation))


 difference from beachmark [pcm]  -154.53638488751585  with standart deviation [pcm]+- 84.63899587712595
